In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import rankdata
from explore import compute_chatterjee_corr_df, compute_chatterjee_corr_np, split_dataframe_k_folds
import statsmodels.api as sm

In [2]:
df = pd.read_csv(r"C:\Users\meeta\OneDrive\Documents\py_projects\soccer_simple_reg\data\raw/NBA_Dataset_csv.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [5]:
df = df.rename(
    columns={
        "Points_Scored": "Points",
        "Weightlifting_Sessions_Average": "WL",
        "Yoga_Sessions_Average": "Yoga",
        "Laps_Run_Per_Practice_Average": "Laps",
        "Water_Intake": "WI",
        "Players_Absent_For_Sessions": "PAFS"
    }
)

In [ ]:
df

In [ ]:
sns.displot(df["Points"], kde=True, stat="density")

plt.show()

In [ ]:
plt.hist(df["Points"], bins=15, color="skyblue", edgecolor="black", density=True)
plt.title('Distribution of Points')
plt.xlabel('Points')
plt.ylabel('Frequency')
sns.kdeplot(data=df["Points"], color="red", linewidth=2)
plt.show()

In [ ]:
sns.boxplot(x=df["Points"], color="skyblue")

plt.show()

In [ ]:
sns.violinplot(x=df["Points"], color="skyblue")
plt.show()

In [11]:
def plot_box_violin_plots(df, x, y):
    fig, ax = plt.subplots(1, 3, figsize=(18, 10))
    
    # Get the y-axis limits from the data
    y_min = df[y].min() - 10
    y_max = df[y].max() + 10
    
    fig.suptitle(f"Violin and box plots for variable {y} against {x}")
    
    # Create plots with consistent y-axis limits
    sns.boxplot(data=df[y], ax=ax[0])
    ax[0].set_ylim(y_min, y_max)
    ax[0].tick_params(axis='x', rotation=90)
    
    sns.boxplot(x=x, y=y, data=df, ax=ax[1], hue=x, palette="Set2")
    ax[1].set_ylim(y_min, y_max)
    ax[1].tick_params(axis='x', rotation=90)
    
    sns.violinplot(x=x, y=y, data=df, ax=ax[2], hue=x, palette="Set2")
    ax[2].set_ylim(y_min, y_max)
    ax[2].tick_params(axis='x', rotation=90)
    
    plt.tight_layout()
    plt.show()

In [ ]:
for col in df.columns:
    if col == "Team":
        continue
    plot_box_violin_plots(df, x="Team", y=col)

In [13]:
def find_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [ ]:
for col in df.columns:
    if col == "Team":
        continue
    outliers = find_outliers(df, col)
    print(f"Outliers for column {col}:")
    print(outliers)
    print()

In [15]:
df_clean = df.drop(index=[142, 143, 144])

In [16]:
df_clean.loc[df_clean.index == 8, "WL"] = None

In [ ]:
plot_box_violin_plots(df_clean, x="Team", y="WL")

#### Imputation Techniques

In [18]:
df_clean_simple = df_clean.copy()
df_clean_iterative = df_clean.copy()
df_clean_knn = df_clean.copy()

In [ ]:
df_clean.isna().mean() # fraction of null values in each column - same as df_clean.isna().sum() / df_clean.shape[0]

In [ ]:
null_counts = pd.DataFrame(df_clean.isna().sum() / df_clean.shape[0]).rename(columns={0: "Null_Proportion"})
null_counts

In [ ]:
null_counts.plot(kind="bar", color="skyblue", title="Proportion of Missing Values in Each Column")
plt.show()

In [ ]:
df_clean.shape, df_clean.dropna().shape

In [ ]:
sns.displot(df_clean["WL"], kde=True, stat="density")
plt.show()

In [ ]:
sns.displot(df_clean["WL"].fillna(df_clean["WL"].mean()), kde=True, stat="density") # naive approach using column mean
plt.show()

In [ ]:
sns.displot(df_clean["WL"].fillna(df_clean["WL"].median()), kde=True, stat="density") # naive approach using column median
plt.show()

In [ ]:
sns.displot(df_clean.fillna(df_clean.loc[:, ["Team", "WL", "Yoga", "Laps", "WI", "PAFS"]].groupby("Team").transform("mean"))["WL"], kde=True, stat="density")
plt.show()

In [ ]:
df_clean.fillna(df_clean.loc[:, ["Team", "WL", "Yoga", "Laps", "WI", "PAFS"]].groupby("Team").transform("mean")) # naive approach but more refined because we are using group means (aggregates)

##### Using Sklearn imputers

In [28]:
features = ["WL", "Yoga", "Laps", "WI", "PAFS"]
simple_imputer = SimpleImputer(strategy="mean")
df_clean_simple.loc[:, features] = simple_imputer.fit_transform(df_clean_simple.loc[:, features])

In [29]:
iterative_imputer = IterativeImputer(max_iter=10)
df_clean_iterative.loc[:, features] = iterative_imputer.fit_transform(df_clean_iterative.loc[:, features])

In [30]:
knn_imputer = KNNImputer(n_neighbors=5)
df_clean_knn.loc[:, features] = knn_imputer.fit_transform(df_clean_knn.loc[:, features])

In [ ]:
df_clean_simple.loc[df_clean.loc[df_clean["WL"].isna()].index]

In [ ]:
df_clean_iterative.loc[df_clean.loc[df_clean["WL"].isna()].index]

In [ ]:
df_clean_knn.loc[df_clean.loc[df_clean["WL"].isna()].index]

In [ ]:
df_clean_iterative

#### Univariate analysis

In [ ]:
featrues = ["WL", "Yoga", "Laps", "WI", "PAFS"]

fig, ax = plt.subplots(2, 3, figsize=(16, 8))

sns.histplot(df_clean_iterative["WL"], kde=True, stat="density", ax=ax[0, 0])
ax[0, 0].set_title("WL - Iterative Imputer")

sns.histplot(df_clean_iterative["Yoga"], kde=True, stat="density", ax=ax[0, 1])
ax[0, 1].set_title("Yoga - Iterative Imputer")

sns.histplot(df_clean_iterative["Laps"], kde=True, stat="density", ax=ax[0, 2])
ax[0, 2].set_title("Laps - Iterative Imputer")

sns.histplot(df_clean_iterative["WI"], kde=True, stat="density", ax=ax[1, 0])
ax[1, 0].set_title("WI - Iterative Imputer")

sns.histplot(df_clean_iterative["PAFS"], kde=True, stat="density", ax=ax[1, 1])
ax[1, 1].set_title("PAFS - Iterative Imputer")

plt.tight_layout()
plt.show()


#### Exploring different values of bandwidth for kernel density estimation (lower values capture too much noise from individual data points and higher values end up over-smoothening sampled dataset thereby losing information)

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(16, 8))
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 0.25}, ax=ax[0, 0])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 0.5}, ax=ax[0, 1])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 0.75}, ax=ax[0, 2])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 0.95}, ax=ax[1, 0])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 1}, ax=ax[1, 1])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 1.25}, ax=ax[1, 2])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 1.5}, ax=ax[2, 0])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 1.75}, ax=ax[2, 1])
sns.histplot(df_clean_knn["WL"], kde=True, stat="density", kde_kws={"bw_adjust": 2}, ax=ax[2, 2])
plt.show()

In [ ]:
df_clean_iterative.corr(numeric_only=True)

In [ ]:
for i in df["Team"].unique():
    print("--------------------")
    print(f"Correlation matrix for team {i}")
    display(df_clean_iterative.loc[df_clean_iterative["Team"] == i].corr(numeric_only=True))
    print("--------------------")

In [ ]:
sns.pairplot(df_clean_iterative, kind="scatter", hue="Team")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (12, 8)
sns.heatmap(df_clean_iterative.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.show()

#### Computation of Chatterjee Correlation Coefficient which verifies non-linear dependence (function) between two variables

In [ ]:
compute_chatterjee_corr_df(df_clean_iterative, x="WL", y="Points")

In [ ]:
compute_chatterjee_corr_np(df_clean_iterative["WL"].values, df_clean_iterative["Points"].values)

In [ ]:
compute_chatterjee_corr_df(df_clean_iterative, x="WI", y="Points")

In [ ]:
compute_chatterjee_corr_np(df_clean_iterative["WI"].values, df_clean_iterative["Points"].values)

In [ ]:
compute_chatterjee_corr_df(df_clean_iterative, x="Laps", y="Points")

In [ ]:
compute_chatterjee_corr_np(df_clean_iterative["Laps"].values, df_clean_iterative["Points"].values)

In [ ]:
compute_chatterjee_corr_df(df_clean_iterative, x="PAFS", y="Points")

In [ ]:
compute_chatterjee_corr_np(df_clean_iterative["PAFS"].values, df_clean_iterative["Points"].values)

In [ ]:
compute_chatterjee_corr_df(df_clean_iterative, x="Yoga", y="Points")

In [ ]:
compute_chatterjee_corr_np(df_clean_iterative["Yoga"].values, df_clean_iterative["Points"].values)

In [52]:
one_hot_df = pd.get_dummies(data=df_clean, columns=["Team"], dtype="int", drop_first=True)

In [53]:
X = one_hot_df.drop(columns=["Points"])
y = one_hot_df["Points"]

In [ ]:
one_hot_df.shape

In [ ]:
folds = split_dataframe_k_folds(df=one_hot_df, k=10)
train_mse = []
test_mse = []
for deg in range(1, 10):
    train_deg_mse = []
    test_deg_mse = []
    for df_k in folds:
        # create a dataframe unioning all the folds except the current one
        df_train = pd.concat([df for df in folds if not df.equals(df_k)])
        X_train = df_train.drop(columns=["Points"])
        y_train = df_train["Points"]
        X_test = df_k.drop(columns=["Points"])
        y_test = df_k["Points"]

        features = X_train.columns
        iterative_imputer = IterativeImputer(max_iter=10)
        X_train.loc[:, features] = iterative_imputer.fit_transform(X_train)
        X_test.loc[:, features] = iterative_imputer.transform(X_test)

        poly_converter = PolynomialFeatures(degree=deg, include_bias=False)
        X_train_poly = poly_converter.fit_transform(X_train)

        model = LinearRegression()
        model.fit(X_train_poly, y_train)
        y_pred = model.predict(poly_converter.transform(X_test))
        
        train_deg_mse.append(mean_squared_error(y_train, model.predict(X_train_poly)))
        test_deg_mse.append(mean_squared_error(y_test, y_pred))
    train_mse.append(
        (deg, train_deg_mse, np.mean(train_deg_mse))
    )
    test_mse.append(
        (deg, test_deg_mse, np.mean(test_deg_mse))
    )

In [ ]:
train_mse

In [ ]:
test_mse

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [56]:
features = X_train.columns
iterative_imputer = IterativeImputer(max_iter=10)
X_train.loc[:, features] = iterative_imputer.fit_transform(X_train)

In [ ]:
X_train

#### OLS with intercept using statsmodels

In [ ]:
lr1 = sm.OLS(y_train, sm.add_constant(X_train)).fit()
lr1.summary()

#### OLS without intercept using statsmodels

In [ ]:
lr2 = sm.OLS(y_train, X_train).fit()
lr2.summary()

#### Model fit with intercept using scikit learn

In [ ]:
model = LinearRegression()
model.fit(X=X_train, y=y_train)

In [61]:
X_test.loc[:, features] = iterative_imputer.transform(X_test)

In [62]:
y_test_pred = model.predict(X_test)

In [ ]:
y_test_pred.shape, y_test.shape

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
df["Points"].hist(bins=15, color="skyblue", edgecolor="black", density=True, ax=ax)
plt.show()

In [ ]:
df["Points"].mean(), df["Points"].median()

In [ ]:
mean_absolute_error(y_test, lr1.predict(sm.add_constant(X_test))) # MAE for model with predictions using statsmodels  api

In [ ]:
mean_absolute_error(y_test, y_test_pred) # MAE for model with predictions using sklearn api

In [ ]:
np.sum(np.abs(y_test - y_test_pred)) / y_test.shape[0] # MAE for model using numpy manual computations and predictions from sklearn api

In [ ]:
mean_squared_error(y_test, lr1.predict(sm.add_constant(X_test))) # MSE for model with predictions using statsmodels api

In [ ]:
mean_squared_error(y_test, y_test_pred) # MSE for model with predictions using sklearn api

In [ ]:
np.sum((y_test - y_test_pred) ** 2) / y_test.shape[0] # mean squared error for model using numpy manual computations and predictions sklearn api

In [ ]:
root_mean_squared_error(y_test, lr1.predict(sm.add_constant(X_test))) # RMSE for model with predictions using statsmodels api

In [ ]:
root_mean_squared_error(y_test, y_test_pred) # RMSE for model with predictions using sklearn api

In [ ]:
np.sqrt(np.sum((y_test - y_test_pred) ** 2) / y_test.shape[0]) # root mean squared error for model using numpy manual computations and predictions sklearn api

In [ ]:
root_mean_squared_error(y_test, y_test_pred) / np.mean(df["Points"]) # relative root mean squared error (relative to the mean of the target variable)

In [ ]:
root_mean_squared_error(y_test, lr1.predict(sm.add_constant(X_test))) / np.mean(df["Points"]) # relative root mean squared error for model with predictions using statsmodels api (relative to the mean of the target variable)

In [ ]:
lr1.rsquared, lr1.rsquared_adj, r2_score(y_train, lr1.predict(sm.add_constant(X_train))), r2_score(y_test, lr1.predict(sm.add_constant(X_test)))

#### Polynomial features

##### Polynomial of degree 2 using numpy polyfit as well as statsmodels output summary

In [78]:
polynomial_converter2 = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
polynomial_converter2.fit(X_train[["WL"]])
features = polynomial_converter2.get_feature_names_out(["WL"])
X_train_poly2 = pd.DataFrame(data=polynomial_converter2.transform(X_train[["WL"]]), columns=features, index=X_train.index)
X_train_poly2

In [ ]:
sm.OLS(y_train, sm.add_constant(X_train_poly2)).fit().summary()

In [ ]:
np.polyfit(X_train["WL"], y_train, deg=2)

##### Polynomial of degree 3 using numpy polyfit as well as statsmodels output summary

In [ ]:
polynomial_converter3 = PolynomialFeatures(degree=3, include_bias=False)
polynomial_converter3.fit(X_train[["WL"]])
features = polynomial_converter3.get_feature_names_out(["WL"])
X_train_poly3 = pd.DataFrame(data=polynomial_converter3.transform(X_train[["WL"]]), columns=features, index=X_train.index)
X_train_poly3

In [ ]:
sm.OLS(y_train, sm.add_constant(X_train_poly3)).fit().summary()

In [ ]:
np.polyfit(deg=3, x=X_train["WL"], y=y_train)

In [85]:
linear_poly = np.poly1d(np.polyfit(X_train["WL"], y_train, deg=1))
quadratic_poly = np.poly1d(np.polyfit(X_train["WL"], y_train, deg=2))
cubic_poly = np.poly1d(np.polyfit(X_train["WL"], y_train, deg=3))
quartic_poly = np.poly1d(np.polyfit(X_train["WL"], y_train, deg=4))
fifth_poly = np.poly1d(np.polyfit(X_train["WL"], y_train, deg=5))

In [86]:
values = np.linspace(X_train["WL"].min()-10, X_train["WL"].max()+10, X_train["WL"].shape[0])

In [ ]:
plt.scatter(X_train["WL"], y_train, color="skyblue", label="Training Data")
plt.plot(values, linear_poly(values), color="red", label="Linear Fit")
plt.plot(values, quadratic_poly(values), color="green", label="Quadratic Fit")
plt.plot(values, cubic_poly(values), color="orange", label="Cubic Fit")
plt.plot(values, quartic_poly(values), color="purple", label="Quartic Fit")
plt.plot(values, fifth_poly(values), color="black", label="Fifth Degree Fit")

plt.ylim(y_train.min() - 10, y_train.max() + 50)
plt.xlim(X_train["WL"].min() - 10, X_train["WL"].max() + 10)
plt.xlabel("WL")
plt.ylabel("Points")
plt.legend()

plt.show()

##### Polynomial features over all features

In [ ]:
polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)
polynomial_converter.fit(X_train)

In [ ]:
features = polynomial_converter.get_feature_names_out(input_features=X_train.columns)
features = [feat.replace(" ", "_") for feat in features]
X_train_poly = pd.DataFrame(data=polynomial_converter.fit_transform(X_train), columns=features, index=X_train.index)
X_train_poly

In [ ]:
model = LinearRegression()
model.fit(X_train_poly, y_train)

In [289]:
y_test_pred = model.predict(pd.DataFrame(data=polynomial_converter.transform(X_test), columns=features, index=X_test.index))

In [ ]:
model.intercept_

In [ ]:
model.coef_

In [ ]:
lr_poly = sm.OLS(y_train, sm.add_constant(X_train_poly)).fit()
lr_poly.summary()

In [ ]:
y_pred = lr_poly.predict(sm.add_constant(polynomial_converter.transform(X_test)))
y_pred

In [ ]:
y_test_pred

In [ ]:
root_mean_squared_error(y_test, y_pred) # root mean squared error for polynomial regression model using statsmodels api

In [ ]:
root_mean_squared_error(y_test, y_test_pred) # root mean squared error for polynomial regression model using sklearn api

In [ ]:
r2_score(y_test, y_pred) # R^2 score for polynomial regression model using statsmodels api

In [ ]:
r2_score(y_test, y_test_pred) # R^2 score for polynomial regression model using sklearn api